In [1]:
from bs4 import BeautifulSoup

import requests
from requests_html import HTMLSession
import html5lib
import pandas as pd
import re


pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)





Scraping indeed job boards to obtain a data set which can be used for analysis

In [2]:
# Creating the connection

# Indeed search for data analyst jobs, displaying 50 per page

URL ="https://www.indeed.com/jobs?as_and=data%20analyst&as_phr&as_any&as_not&as_ttl&as_cmp&jt=all&st&salary&radius=25&l&fromage=any&limit=50&sort&psf=advsrch&from=advancedsearch&vjk=44dce43f1d302551&advn=5680616533559579"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 "     
          }  

response = requests.get(URL,headers=headers)

content = response.content

# print(content)



In [3]:
# Using beautifulsoup to parse the HTMML


soup1=BeautifulSoup(response.content, 'html.parser')
soup = BeautifulSoup(soup1.prettify(), 'html.parser')





In [4]:
# Cleaning date data to extract only numbers.

DatePosted = soup.find_all("span", class_="date")
for dates in DatePosted:
    content = dates.text.replace('\n','').strip()
    content = content.replace('Employer','')
    content = content.replace('Posted','')
    content = content.replace('Active','')
    content = content.replace('ago','')
    content = content.replace('days','')
    content = content.replace('day','')
    content = content.replace('Just posted','0')     # Consider jobs posted today as 0 days ago
    content = content.replace('To','0')
    content = content.replace('30+','30')            # Jobs Posted 30+ days ago as 30 days ago
    content = re.sub("^\s+|\s+$", "", content, flags=re.UNICODE)
    print(content)
    
    

NameError: name 're' is not defined

In [ ]:
Company_Names=[] 
Dates_Posted = []
Job_Titles = []
Company_Locations=[]
Job_Salaries = []


# Creating lists and turning the list data into a datframe and cleaning up string and date data.



CompanyNames = soup.find_all("span",class_="companyName")
for name in CompanyNames:
    content = name.text.strip().replace(".", "")
    editedSource = re.sub('\â€”','',str(content))
    Company_Names.append(content)


DatePosted = soup.find_all("span", class_="date")
for dates in DatePosted:
    content = dates.text.replace('\n','').strip()
    content = content.replace('Employer','')
    content = content.replace('Posted','')
    content = content.replace('Active','')
    content = content.replace('ago','')
    content = content.replace('days','')
    content = content.replace('day','')
    content = content.replace('Just posted','0')     # Consider jobs posted today as 0 days ago
    content = content.replace('To','0')              # day removed, jobs posted today 'to' set as 0
    content = content.replace('30+','30')            # Jobs Posted 30+ days ago as 30 days ago
    content = re.sub("^\s+|\s+$", "", content, flags=re.UNICODE)
    Dates_Posted.append(content)
    
    
    
Titles = soup.find_all("h2", class_="jobTitle")
for title in Titles:
    data = title.text.replace('\n','').strip()
    data = data.replace('new','')
    data = re.sub(r"^\s+", "", data, flags=re.UNICODE)
    data = re.sub('\â€”','',str(data))
    Job_Titles.append(data)



CompanyLocation = soup.find_all("div",class_="companyLocation")
for loca in CompanyLocation:
    dataloc = loca.text.replace('\n','').strip()
    dataloc = dataloc.replace('(','')
    dataloc = dataloc.replace(')','')
    dataloc = dataloc.replace('+','')
    dataloc = dataloc.replace('locations','')
    dataloc = dataloc.replace('location','')
    dataloc = dataloc.replace('  ','')
    dataloc = re.sub('\d', '', dataloc)
    dataloc = re.sub("^\s+|\s+$", "", dataloc, flags=re.UNICODE)
    Company_Locations.append(dataloc)
    

    
    
    
#SalaryRange = soup.find_all("div",class_="metadata salary-snippet-container")
#for sal in SalaryRange:
#    Job_Salaries.append((sal.text.strip()))

   


In [ ]:


df = pd.DataFrame( 
{  'Job Title' : Job_Titles,
    'Company Name': Company_Names,
    'Company Location': Company_Locations,
    'Days Posted Ago': Dates_Posted,
 #   'Salary Range': Job_Salaries}
}, columns=["Job Title", "Company Name", "Company Location", "Days Posted Ago"]
)


print(df)

df.to_csv('IndeedScraping.csv', index=False)




In [ ]:
 # Function to automate this process - 

def scrap_indeed():
    import datetime
    import csv
    from bs4 import BeautifulSoup
    import requests
    import requests
    from requests_html import HTMLSession
    import html5lib
    import pandas as pd
    import re
    
    URL ="https://www.indeed.com/jobs?as_and=data%20analyst&as_phr&as_any&as_not&as_ttl&as_cmp&jt=all&st&salary&radius=25&l&fromage=any&limit=50&sort&psf=advsrch&from=advancedsearch&vjk=44dce43f1d302551&advn=5680616533559579"

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36 "     
          }  

    response = requests.get(URL,headers=headers)
    content = response.content
    soup1=BeautifulSoup(response.content, 'html.parser')
    soup = BeautifulSoup(soup1.prettify(), 'html.parser')

    Company_Names=[] 
    Dates_Posted = []
    Job_Titles = []
    Company_Locations=[]
    Job_Salaries = []


    CompanyNames = soup.find_all("span",class_="companyName")
    for name in CompanyNames:
        content = name.text.strip().replace(".", "")
        editedSource = re.sub('\â€”','',str(content))
        Company_Names.append(content)


    DatePosted = soup.find_all("span", class_="date")
    for dates in DatePosted:
        content = dates.text.replace('\n','').strip()
        content = content.replace('Employer','')
        content = content.replace('Posted','')
        content = content.replace('Active','')
        content = content.replace('ago','')
        content = content.replace('days','')
        content = content.replace('day','')
        content = content.replace('Just posted','0')     # Consider jobs posted today as 0 days ago
        content = content.replace('To','0')              # day removed, jobs posted today 'to' set as 0
        content = content.replace('30+','30')            # Jobs Posted 30+ days ago as 30 days ago
        content = re.sub("^\s+|\s+$", "", content, flags=re.UNICODE)
        Dates_Posted.append(content)
    
    
    
    Titles = soup.find_all("h2", class_="jobTitle")
    for title in Titles:
        data = title.text.replace('\n','').strip()
        data = data.replace('new','')
        data = re.sub(r"^\s+", "", data, flags=re.UNICODE)
        data = re.sub('\â€”','',str(data))
        Job_Titles.append(data)



    CompanyLocation = soup.find_all("div",class_="companyLocation")
    for loca in CompanyLocation:
        dataloc = loca.text.replace('\n','').strip()
        dataloc = dataloc.replace('(','')
        dataloc = dataloc.replace(')','')
        dataloc = dataloc.replace('+','')
        dataloc = dataloc.replace('locations','')
        dataloc = dataloc.replace('location','')
        dataloc = dataloc.replace('  ','')
        dataloc = re.sub('\d', '', dataloc)
        dataloc = re.sub("^\s+|\s+$", "", dataloc, flags=re.UNICODE)
        Company_Locations.append(dataloc)
    
    
    df = pd.DataFrame( 
    {  'Job Title' : Job_Titles,
        'Company Name': Company_Names,
        'Company Location': Company_Locations,
        'Days Posted Ago': Dates_Posted,}
    , columns=["Job Title", "Company Name", "Company Location", "Days Posted Ago"]
    )

    
    with open('IndeedScraping.csv', 'a+', newline='', encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)

    

In [ ]:
import time

while(True):
    scrap_indeed()
    time.sleep(1000)
    